In [ ]:
# Installation (please use latest versions if stucked anywhere)
!pip install -q accelerate==0.34.2
!pip install -q peft==0.14.0
!pip install -q bitsandbytes==0.45.0
!pip install -q transformers==4.47.1
!pip install -q trl==0.13.0
#!pip install -q fastrlock==0.8.2

In [ ]:
# Imports
import os, torch, logging
from datasets import load_dataset, concatenate_datasets, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    EarlyStoppingCallback)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
# set enironment variables (secrets if working in colab)
os.environ['HF_TOKEN'] = '<your-huggingface-token>'
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# below is the huggingface directory location to load the model from
# you can download the model in local and give the location to that as well
finetuned_model_name = "amanpreetsingh459/gemma-2-2b-punjabi-finetuned-4"

In [ ]:
gemma_tokenizer = AutoTokenizer.from_pretrained(finetuned_model_name)
EOS_TOKEN = gemma_tokenizer.eos_token

In [ ]:
model_finetuned = AutoModelForCausalLM.from_pretrained(
    finetuned_model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

In [ ]:
alpaca_prompt = """
### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
inputs = gemma_tokenizer(
[
    alpaca_prompt.format(
        "ਮੇਨੂ ਏਕ ਕਵਿਤਾ ਲਿੱਖ ਕੇ ਦੇਯੋ ਜੀ", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

In [ ]:
outputs = model_finetuned.generate(**inputs, max_new_tokens = 250)

In [ ]:
decoded_outputs = gemma_tokenizer.batch_decode(outputs)
print(decoded_outputs[0])